In [1]:
#!/usr/bin/env python3
import torch
from torch import nn
import numpy as np

In [2]:
class MyNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(1, 2)
        self.output = nn.Linear(2, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.output(x)
        return x

x = torch.tensor([[0], [1]], dtype=torch.float32)
d = torch.tensor([[2], [4]], dtype=torch.float32)

sigmoid = nn.Sigmoid()

net = MyNetwork()

v0 = 0
v1 = 4
v2 = -1.0

w11 = 2.0
w10 = 0

w21 = 0.1
w20 = 1.0

# learning rate
eta = 0.1

w = torch.tensor([v0, v1, v2, w10, w20, w11, w21])

net.hidden.weight.data = torch.tensor([[w[5]], [w[6]]])
net.hidden.bias.data = torch.tensor([w[3], w[4]])

net.output.weight.data = torch.tensor([[w[1], w[2]]])
net.output.bias.data = torch.tensor([w[0]])

# Calculate the error
sum = 0
for i in range(x.size()[0]):
    sum = sum + pow((net(x[i]) - d[i]), 2)
E = 0.5*sum

net(x[0]), net(x[1]), E

(tensor([1.2689], grad_fn=<AddBackward0>),
 tensor([2.7729], grad_fn=<AddBackward0>),
 tensor([1.0201], grad_fn=<MulBackward0>))

In [3]:
epochs = 1

for i in range(epochs):    
    # Calculate the gradients
    dE_dv1 = 0;
    sum = 0;
    for i in range(0, 2):
        z1 = sigmoid(x[i]*w11 + w10)
        sum = sum + (net(x[i]) - d[i])*z1
    dE_dv1 = sum

    dE_dv2 = 0;
    sum = 0;
    for i in range(0, 2):
        z2 = sigmoid(x[i]*w21 + w20)
        sum = sum + (net(x[i]) - d[i])*z2
    dE_dv2 = sum

    dE_dv0 = 0;
    sum = 0;
    for i in range(0, 2):
        sum = sum + (net(x[i]) - d[i])
    dE_dv0 = sum

    dE_dw11 = 0;
    sum = 0;
    for i in range(0, 2):
        sum = sum + (net(x[i]) - d[i])*v1*x[i]*sigmoid(x[i]*w11 + w10)*(1-sigmoid(x[i]*w11 + w10))
    dE_dw11 = sum

    dE_dw10 = 0;
    sum = 0;
    for i in range(0, 2):
        sum = sum + (net(x[i]) - d[i])*v1*sigmoid(x[i]*w11 + w10)*(1-sigmoid(x[i]*w11 + w10))
    dE_dw10 = sum

    dE_dw21 = 0;
    sum = 0;
    for i in range(0, 2):
        sum = sum + (net(x[i]) - d[i])*v2*x[i]*sigmoid(x[i]*w21 + w20)*(1-sigmoid(x[i]*w21 + w20))
    dE_dw21 = sum

    dE_dw20 = 0;
    sum = 0;
    for i in range(0, 2):
        sum = sum + (net(x[i]) - d[i])*v2*sigmoid(x[i]*w21 + w20)*(1-sigmoid(x[i]*w21 + w20))
    dE_dw20 = sum

    # Get the Gradient vector
    dE_dw = torch.tensor([dE_dv0, dE_dv1, dE_dv2, dE_dw10, dE_dw20, dE_dw11, dE_dw21])

    # Update the weights
    w = w - eta*dE_dw

    # Update the network
    net.hidden.weight.data = torch.tensor([[w[5]], [w[6]]])
    net.hidden.bias.data = torch.tensor([w[3], w[4]])

    net.output.weight.data = torch.tensor([[w[1], w[2]]])
    net.output.bias.data = torch.tensor([w[0]])

    print(net(x[0]).data, net(x[1]).data, dE_dw, w)

tensor([1.7788]) tensor([3.2868]) tensor([-1.9581, -1.4463, -1.4551, -1.2464,  0.3737, -0.5153,  0.2299]) tensor([ 0.1958,  4.1446, -0.8545,  0.1246,  0.9626,  2.0515,  0.0770])
